In [1]:
from mess2014 import show_distance_plot, array_analysis_helper, align_phases, array_transfer_helper, vespagram

# Introduction to Array Seismology Methods

 #### 1. Seismograms, move-out
 #### 2. Vespagrams, linear and 4th root
 #### 3. Phase Weighted Stack (PWS)
 #### 4. Array Response Function
 #### 5. Slowness-Backazimuth plots
 #### 6. Frequency-Wavenumaber analysis

This practical introduces the most common array processing techniques used for array seismology studies. 
We will introduce standard techniques and use them on selected examples from an event recorded at the German Regional Seismic Network (GRSN) and a recently deployed array in Morocco (Muenster Morocco array, MM) and teleseismic events.


           



   #### Information for each event
<table>
       <tr>
        <th>#</th> <th>date</th> <th>time</th> <th>lat</th> <th>lon</th> <th>depth(km)</th> <th>magn</th> <th>array</th> <th>phase</th>
        </tr>
        <tr>
            <td>1</td> <td>17 DEC 1991</td> <td>06:38:17</td> <td>47.429</td> <td>151.484</td> <td>150</td> <td>5.5</td> <td>GRF,GRSN</td>     <td>P, PcP</td>
        </tr>
        <tr>
            <td>2</td> <td>14 AUG 2012</td> <td>02:59:38</td> <td>49.80</td> <td>145.064</td> <td>583</td> <td>7.7</td> <td>MM</td>     <td>P, PP</td>
        </tr>
        <tr>
            <td>3</td> <td>14 AUG 2012</td> <td>02:59:38</td> <td>49.80</td> <td>145.064</td> <td>583</td> <td>7.7</td> <td>YKA</td>     <td>diff, PP</td>
        </tr>
        <tr>
            <td>4</td> <td>05 MAY 2003</td> <td>23:04:45</td> <td>3.645</td> <td>128.010</td> <td>61</td> <td>6.2</td> <td>KNET</td>     <td>mislo, pP</td>
        </tr>
        <tr>
            <td>5</td> <td>04 OCT 2002</td> <td>19:05:50</td> <td>-20.96</td> <td>-178.86</td> <td>630</td> <td>6.3</td> <td>EAGLE</td>     <td>scattering</td>
           </tr>
        <tr>
        
</table>        
Please keep in mind that the **data** are **accesible** at **/home/mess/Desktop/data**.

For example for the Morocco event: /home/mess/Desktop/data/02_MM/02.mseed

### **Getting Genral information of each event and array

**a)** Plot the morocco event (14 AUG 2012) on a world map (using QuakeML file)


In [ ]:
import obspy

# Event information is stored in the QuakeML file.
cat = obspy.read_events("./02_MM/02.qml")
print cat
cat.plot()

**b)** Print the stations informations of MM array (using stationXML file)

In [ ]:
# Station information is stored in the StationXML file.
inv = obspy.read_inventory("./02_MM/02.xml", format="stationxml")
print inv

### 1) Seismograms and Move-out of different waves.

Read the Morocco data (event 2). The data have been cut to a start time of 14th August 2012 2:30 for 5400 s

In this event we are concentrating on the time range between the PcP wave and the PP wave.

**a)** Display the data as a normal seismogram section (all data, z-component only). 

Use st.select to select Z-components.


In [ ]:
import obspy
st = obspy.Stream()
st.select?

In [ ]:
import obspy

# The first step is to read in the data.
st = obspy.read("./02_MM/02.mseed")
vertical_components = st.select(component="Z")
vertical_components.plot()


**b)** Often a distance dependent plot shows the different move-out of seismic arrivals and gives an idea of the slowness that could be expected. Produce a distance dependent plot of the data, use the time of 3:11:00 to 3:18:00 for the Morocco event (event 2) (if possible with phases marked - note P and PcP arrive at almost the same time)

use show_distance_plot
(with catalog and inventory from above).

In [ ]:
show_distance_plot?

In [ ]:
import obspy
show_distance_plot(vertical_components, cat[0], inv, obspy.UTCDateTime('2012-08-14T3:11:00'), obspy.UTCDateTime('2012-08-14T3:18:00'), plot_travel_times=True)

**c)** As it is difficult to see different move-outs if the data are not aligned on a single phase, it is often useful to align. Align the data on the first arrival (P) and the different  move-out for each of the phases becomes visible. 

Use align_phases and asign them to a stream st (st=align_phases...), 
take only the z-component and catalog and inventory from above)

plot traces again as distance dependent plot (show_distance_plot)

In [ ]:
align_phases?

In [ ]:
ap = align_phases(vertical_components, cat[0], inv,'P', method='simple')
show_distance_plot(ap, cat[0], inv, obspy.UTCDateTime('2012-08-14T3:11:00'), obspy.UTCDateTime('2012-08-14T3:18:00'), plot_travel_times=False)

**d)** To see the move-out difference in more detail zoom in to the PP wave. Use 900 to 1100 s after origin time.

In [ ]:
origin_time = cat[0].origins[0].time
t1 = origin_time + 900
t2 = origin_time + 1100
show_distance_plot(ap, cat[0], inv, t1, t2, plot_travel_times=False)


**e)** Filter events with different filters, best suited is bandpass filter 1-10s (teleseismic P-waves). Align the P wave and display a time window including the P and PP wave. Use times from 740 to 1150 s after origin.


In [ ]:
origin_time = cat[0].origins[0].time
t1 = origin_time + 740
t2 = origin_time + 1150
st = ap.copy()
st.filter('bandpass',freqmin=0.1,freqmax=1.)
show_distance_plot(st, cat[0], inv, t1, t2, plot_travel_times=False)

 - For the **first event**  in the table above (17 DEC1991) we are concentrating on the P and PcP wave:

The time range here is 17-Dec-91_6:49:30 to 6:50:30.

Plot the event on a world map.



In [ ]:
import obspy
stream_1 = obspy.read("/home/mess/Desktop/data/01_GRSN_GRF/01.mseed")
cat_1 = obspy.readEvents("/home/mess/Desktop/data/01_GRSN_GRF/01.qml")
inv_1 = obspy.read_inventory("/home/mess/Desktop/data/01_GRSN_GRF/01.xml", format="stationxml")

cat_1.plot()

Plot the data in a distance dependent plot with the traces aligned on the P-wave.

This event (17 Dec 1991) shows the P-arrival and a secondary arrival with large amplitude.
The distance dependent plot with traces aligned on the P-arrival shows that the secondary arrival has a different move-out than the P-wave. It is perhaps useful to zoom in (6:49:30 to 6:50:30).
Use cat_1 and inv_1 for catalog and inventory.

In [ ]:
import obspy
from mess2014 import show_distance_plot, array_analysis_helper, align_phases, array_transfer_helper, vespagram
t1 = obspy.UTCDateTime('1991-12-17T06:49:30')
t2 = obspy.UTCDateTime('1991-12-17T06:50:30')
vert = stream_1.select(component='Z')
ap = align_phases(vert, cat_1[0], inv_1,'P', method='simple')
show_distance_plot(ap, cat_1[0], inv_1, t1, t2, plot_travel_times=False)

## Reminder: Calculating theoretical backazimuth and slowness

From the delay times measured at each station with respect to the centre of the array, the slowness and backazimuth of a phase can be determined. The horizontal slowness $u_{hor}$ and backazimuth $\Phi$ are calculated, assuming all receivers are sea level and a plane wave arrival, using the following formula (Schweitzer et al., 2002):

$$ t_j = \frac{d_j}{v_{app}} = \frac{-x_j  \sin \Phi - y_j \cos \Phi}{v_{app}}$$

 

$$ v_{app} = \frac{-x_j  \sin \Phi - y_j \cos \Phi}{t_j} $$

<img src="http://earth.uni-muenster.de/~msaki_01/figure1.png">

Beamforming could be used here but the process is lengthy and tedious and helps to enhance only the phase with a known backazimuth and slowness. Wrong backazimuths could also introduce misleading waveforms. It is important to note that the delay times could be influences by the subsurface and could provide erroneous stacks (more later).

Skipping the beamforming we will proceed to vespagram analysis.


### 3) Vespagram (velocity spectral analysis)
#### 3a) Perform linear vespagram for theo baz
To make the process easier, a vespagram (velocity spectral analysis) is produced. Here the traces are shifted and stacked for a theoretical backazimuth and a range of slowness-values (or vice versa). A linear stack will look very wiggly and the slowness resolution will not be very good. 

Still it is worth to try for the two events, for large aperture arrays and a large number of stations this may still work well. To perform waveform studies, a linear stack (vespagram) is imperative.

Read in the Morocco data again:

The azimuth of the centre of the array is used (17 deg). Try different slowness ranges  between 0 and 10 s/deg for the vespagram but do not exceed the slowness increment of 0.2 to 0.5. 
Time range from 20 sec before Pdiff to 50 s after PP (or absolute times 3:11:40 to 3:17:00)
use the delay and sum (DLS) option and an nthroot of 1 (for the linear stack).
You can try broadband data or different filters (e.g., 0.1 to 4 Hz) and zoom-values (e.g. 10). 
Use the plot_trace=True option wor wiggles.


In [ ]:
vespagram?

In [ ]:
from obspy import read, UTCDateTime, readEvents, read_inventory
st =read("/home/mess/Desktop/data/02_MM/02.mseed")
t1 = UTCDateTime(2012, 8, 14, 3, 11, 40)
t2 = UTCDateTime(2012, 8, 14, 3, 17)
vertical_components = st.select(component="Z")
vertical_components.trim(t1, t2)
print vertical_components

ev = readEvents("/home/mess/Desktop/data/02_MM/02.qml")[0]
inv = read_inventory("/home/mess/Desktop/data/02_MM/02.xml")

from mess2014 import vespagram
vespagram(vertical_components, ev, inv, 'DLS', 1., 4., 17,10, nthroot=4, filter=False, static3D=False, vel_corr=4.8, sl=(0.0, 10.0, 0.5), align=False, align_phase=['P', 'Pdiff'])

Make the vespagram in contour plot (plot_trace=false) - sometimes it looks better in a contour plot

In [ ]:
from obspy import read, UTCDateTime, readEvents, read_inventory
st =read("/home/mess/Desktop/data/02_MM/02.mseed")
t1 = UTCDateTime(2012, 8, 14, 3, 11, 40)
t2 = UTCDateTime(2012, 8, 14, 3, 17)
vertical_components = st.select(component="Z")
vertical_components.trim(t1, t2)
print vertical_components

ev = readEvents("/home/mess/Desktop/data/02_MM/02.qml")[0]
inv = read_inventory("/home/mess/Desktop/data/02_MM/02.xml")

from mess2014 import vespagram


#### 3b) Perform 4th root vespa for theo baz

To have a better resolution for the arrivals in the vespagram, a 4th root stack should be produced. The waveforms are poorer, the slowness resolution is much better. Both events (event 1 and 2) show clear arrivals, P and PdP (the D" reflection) in the first event can clearly be seen, P/PcP and PP can clearly be distinguished in the second example.

The first exercise is for the event recorded at the Morocco stations (event2)

Use the same parameters as for the linear vespagram but with a 4th root stacking option (you can try different nthroot values as well).
Try different slowness ranges between 0 and 10 s/deg with increments of 0.2 s/deg. Try different filters. 

In [ ]:
from obspy import read, UTCDateTime, readEvents, read_inventory
st =read("/home/mess/Desktop/data/02_MM/02.mseed")
t1 = UTCDateTime(2012, 8, 14, 3, 11, 40)
t2 = UTCDateTime(2012, 8, 14, 3, 17)
vertical_components = st.select(component="Z")
vertical_components.trim(t1, t2)
print vertical_components

ev = readEvents("/home/mess/Desktop/data/02_MM/02.qml")[0]
inv = read_inventory("/home/mess/Desktop/data/02_MM/02.xml")

from mess2014 import vespagram


### 4) Perform pws for the same event
Phase weighted stacking produces a different kind of enhanced vespagram. To see the difference (or lack of?) PWS of the above events should be produced.
In the vespagram routine use PWS as option. Again try different slowness ranges and filters to get the best result.

In [ ]:
from obspy import read, UTCDateTime, readEvents, read_inventory
st =read("/home/mess/Desktop/data/02_MM/02.mseed")
t1 = UTCDateTime(2012, 8, 14, 3, 11, 40)
t2 = UTCDateTime(2012, 8, 14, 3, 17)
vertical_components = st.select(component="Z")
vertical_components.trim(t1, t2)
print vertical_components

ev = readEvents("/home/mess/Desktop/data/02_MM/02.qml")[0]
inv = read_inventory("/home/mess/Desktop/data/02_MM/02.xml")

from mess2014 import vespagram


For the **GRSN/GRF** array event, produce a 4th root vespagram and try different slowness values (between 0 and 10 s/deg, increment not exceeding 0.2s/deg), a backazimuth of the centre of the array (24deg) and a time around P and PcP (absolute time 17-dec-91_6:49:30 to 6:50:30). The P-wave is clearly visible, the second large arrival has a smaller slowness and two more arrivals are apparent, one is PcP (approximately 9.5 s after P and another arrivals with PcP slowness. 

In [ ]:
from obspy import read, UTCDateTime, readEvents, read_inventory
st = read("/home/mess/Desktop/data/01_GRSN_GRF/01.mseed")

vertical_components = st.select(component="Z")
t1 = UTCDateTime(1991, 12, 17, 6, 49, 30)
t2 = UTCDateTime(1991, 12, 17, 6, 50, 30)
vertical_components.trim(t1, t2) 

cat = readEvents("/home/mess/Desktop/data/01_GRSN_GRF/01.qml")
ev = cat[0]
inv = read_inventory("/home/mess/Desktop/data/01_GRSN_GRF/01.xml", format='STATIONXML')

from mess2014 import vespagram


### 5) Perform sloaz for the Morocco data with longer time window or moving time window

If slowness and backazimuth of the data are not known (or true, i.e. measured, values should be determined) a sloaz analysis can be produced. Here chose a time window around the Pdiff arrival (or any other arrival that you like) and produce a sloaz analysis. The range of backazimuths depends on the theoretical backazimuth. 

For the Event recorded in Morocco the theoretical backazimuth is 17.3 degrees. The slowness range is determined by the theoretical slowness values of the phase we are interested in. For Pdiff to PP the range should be between 0 and 10 s/deg.

The window that includes Pdiff and PP is between time 3:11:30 to 3:14:30.
Produce slowness_backazimuth plot for two windows, one around Pdiff, one around PP (hint: import array_analysis_helper routine from mess2014)

Don't plot the array response function here, use method "FK" and baz_plot=True. Also test different values for the slowness (slx, sly) to find the best fitting values.

In [ ]:
array_analysis_helper?

In [ ]:
from obspy import read, UTCDateTime, readEvents, read_inventory
st = read("/home/mess/Desktop/data/02_MM/02.mseed")

vertical_components = st.select(component="Z")
t1 = UTCDateTime(2012, 8, 14, 3, 11, 30)
t2 = UTCDateTime(2012, 8, 14, 3, 14, 30)
vertical_components.trim(t1, t2) 

inv = read_inventory("/home/mess/Desktop/data/02_MM/02.xml", format='STATIONXML')

from mess2014 import array_analysis_helper


### 6) Display array response function in fk and sloaz display

Here we introduce the concept of array response function. The array response function depends on the array configuration and signal frequency.  The best examples are usually Yellowknife data since the ARF is a cross (like the array) and can be recognised very well in the fk plot.

Perform fk analysis for the Yellowknife array. Use default slowness values (you will need to use array_transfer_helper and array_analysis_helper (with baz_plot=False)).

In [ ]:
array_transfer_helper?

In [ ]:
from obspy import read, UTCDateTime, read_events, read_inventory
st = read("03_YKA/03.mseed")


vertical_components = st.select(component="Z")
t1 = UTCDateTime(2012, 8, 14, 3, 6, 30)
t2 = UTCDateTime(2012, 8, 14, 3, 9, 0)
vertical_components.trim(t1, t2) 

inv = read_inventory("03_YKA/03.xml", format='STATIONXML')

from mess2014 import array_transfer_helper, array_analysis_helper


### 7) FK analysis (Frequency-Wavenumber analysis)
#### 7a) perform fk analysis for P-wave

Use the GRF array data only, fk can usually only be done for small time windows to avoid interfering phases. For the GRF data, chose a time window of absolute time 17-dec-9_6:49:55 to 6:50:02).  Try different slowness ranges to find a fitting slowness range (from 0 to 12s/deg). 

In [ ]:
from obspy import read, UTCDateTime, readEvents, read_inventory
st = read("/home/mess/Desktop/data/01_GRSN_GRF/01.mseed")

vertical_components = st.select(component="Z")
t1 = UTCDateTime(1991, 12, 17, 6, 49, 55)
t2 = UTCDateTime(1991, 12, 17, 6, 50, 2)
vertical_components.trim(t1, t2) 

cat = readEvents("/home/mess/Desktop/data/01_GRSN_GRF/01.qml")
ev = cat[0]
inv = read_inventory("/home/mess/Desktop/data/01_GRSN_GRF/01.xml", format='STATIONXML')

from mess2014 import array_analysis_helper


#### 7b) perform fk-analysis for PdP wave

To see the difference to the P-wave, the secondary arrival (it is PdP, the reflection off the D" discontinuity) can be analysed with the fk-analysis: chose a time window from 17-dec-9_6:50:01 to 6:50:06. The slowness range should be as before (in the range of 0-12 s/deg). The slowness values for PdP are smaller (while the backazimuth is very similar to the P wave) and therefore cannot be a depth phase or Moho multiple. If projecting the PdP wave back with the measured slowness, it turns out to be reflected off the D" discontinuity.



In [2]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 8, 8


You can try different times windows as well and see whether you can see PcP as well!

In [3]:
# Load waveforms only once
from obspy import *
from obspy.clients.fdsn import Client

cl = Client("http://jane")
start = UTCDateTime("2022-04-09T06:57:25")
end = start + 10
st = cl.get_waveforms(network="BW",station="SYB0*",location="",channel="EL*",starttime=start,endtime=end)
inv = cl.get_stations(network="BW",station="SYB0*",location="",channel="EL*",starttime=start,endtime=end,level="response")

In [4]:
%matplotlib inline

st.attach_response(inv)
st.remove_response(water_level=10)

vertical_components = st.select(component="Z")


In [5]:
from mess2014 import array_analysis_helper
print (array_analysis_helper(vertical_components, inv, method="FK", frqlow=10,slx=(-100,100), sly=(-100, 100), sls=2,
                            frqhigh=20, baz_plot=True, wlen=0.5,array_response=False))

3 Trace(s) in Stream:
BW.SYB01..ELZ | 2022-04-09T06:57:24.999999Z - 2022-04-09T06:57:34.999999Z | 200.0 Hz, 2001 samples
BW.SYB02..ELZ | 2022-04-09T06:57:24.999998Z - 2022-04-09T06:57:34.999998Z | 200.0 Hz, 2001 samples
BW.SYB03..ELZ | 2022-04-09T06:57:24.999999Z - 2022-04-09T06:57:34.999999Z | 200.0 Hz, 2001 samples
Total time in routine: 0.095199



FileNotFoundError: [Errno 2] No such file or directory: '/var/folders/xz/s5p82pmd36x87cv7pcmc769w0000gn/T/obspy-708xz2z2/pow_map_001.npy'